In [1]:
import pandas as pd
import numpy as np
import torch
import random
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, average_precision_score
import evaluate
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
import numpy as np
from sklearn.impute import KNNImputer
import os
os.environ["WANDB_DISABLED"] = "true"


# Function to set seed for reproducibility
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed()

# Disable wandb
os.environ["WANDB_DISABLED"] = "true"

# GPU or CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Ensure PyTorch uses all available threads
torch.set_num_threads(torch.get_num_threads())

def data_pre_processing(df):
    timestamp_columns = [col for col in df.columns if "timestamp" in col]
    other_non_imp_column_to_remove = ["id", "general_relapse_class"]
    columns_with_all_nan = df.columns[df.isna().all(axis=0)].tolist()
    print("shape of the dataframe before dropping columns", df.shape)
    df.drop(timestamp_columns + other_non_imp_column_to_remove + columns_with_all_nan, axis=1, inplace=True)
    print("shape of the dataframe after dropping columns", df.shape)
    categorical = df.select_dtypes(include=['bool', 'object']).columns.tolist()
    numerical = df.select_dtypes(include=['float64', 'int64']).columns.tolist()
    print("Length of categorical and numerical columns:", len(categorical), len(numerical))
    imputer = KNNImputer(n_neighbors=5, weights='uniform')
    imputed_data = imputer.fit_transform(df[numerical])
    df_imputed_numerical = pd.DataFrame(imputed_data, columns=numerical, index=df.index)
    df[numerical] = df_imputed_numerical
    return pd.get_dummies(df, columns=categorical, drop_first=True)

# In[3]:

df = pd.read_csv("/home/ubuntu/Tabular_Machine_Learning_Using_LLM/Data/patient_features_early_stage.csv")
df.rename(columns={'relapse?': 'general_relapse_class'}, inplace=True)
y = df["general_relapse_class"].values
df_encoded = data_pre_processing(df)
X = df_encoded.values
df_encoded.head()

def row_to_sentence_full(row):
    sentence_parts = [
        f"A patient of age {row['age']} started chemotherapy at {row['chemotherapy@t1_start_time']} days",
        f"underwent radiotherapy for {row['radiotherapy@t1_duration_days']} days",
        f"and had surgery at {row['surgery@t1_time']} days",
        f"with a history of smoking {row['nb_cig_packs_year']} cigarette packs per year",
        f"and {row['nb_cigs_day']} cigarettes per day",
    ]
    family_history = []
    if row['family_lung_cancer']:
        family_history.append("lung cancer")
    if row['family_other_cancer']:
        family_history.append("other types of cancer")
    if family_history:
        sentence_parts.append(f"and a family history that includes: {', '.join(family_history)}")
    else:
        sentence_parts.append("and no family history of cancer")
    for col in row.index:
        if pd.api.types.is_numeric_dtype(row[col]) and not '@' in col and col not in ['age', 'Unnamed: 0']:
            sentence_parts.append(f"{col.replace('_', ' ')} of {row[col]}")
        elif pd.api.types.is_bool_dtype(row[col]) and row[col]:
            sentence_parts.append(f"and has a condition of {col.replace('@', ' at ').replace('_', ' ')}")
    sentence = ', '.join(sentence_parts) + '.'
    return sentence


texts = df_encoded.apply(row_to_sentence_full, axis=1).tolist()
labels = y.tolist()

data = {'text': texts,
        'label':labels}
new_df = pd.DataFrame(data)

X = new_df["text"]
y = new_df["label"]

relapse_df = new_df[new_df['label'] == 1]
non_relapse_df = new_df[new_df['label'] == 0]

# For relapse patients
relapse_train, relapse_test = train_test_split(relapse_df, test_size=100, random_state=42)
relapse_train, relapse_val = train_test_split(relapse_train, test_size=50, random_state=42)

# For non-relapse patients
non_relapse_train, non_relapse_test = train_test_split(non_relapse_df, test_size=100, random_state=42)
non_relapse_train, non_relapse_val = train_test_split(non_relapse_train, test_size=50, random_state=42)

train_df = pd.concat([relapse_train, non_relapse_train])
test_df = pd.concat([relapse_test, non_relapse_test])
val_df = pd.concat([relapse_val, non_relapse_val])

train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)
val_dataset = Dataset.from_pandas(val_df)


tokenizer = AutoTokenizer.from_pretrained('EleutherAI/gpt-neo-125M')
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# Preprocess function adjusted for Hugging Face Datasets
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True, padding=True, max_length=512)

# tokenized_text = data.map(preprocess_function, batched = True)
tokenized_train = train_dataset.map(preprocess_function, batched = True)
tokenized_val = val_dataset.map(preprocess_function, batched = True)
tokenized_test = test_dataset.map(preprocess_function, batched = True)


# Model
model = AutoModelForSequenceClassification.from_pretrained('EleutherAI/gpt-neo-125M', num_labels=2).to(device)
model.config.pad_token_id = tokenizer.pad_token_id
# Training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=50,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    warmup_steps=500,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    logging_dir='./logs',
)

# Accuracy metric
accuracy = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    compute_metrics=compute_metrics,
)

# Train the model
trainer.train()
logits, _, _ = trainer.predict(tokenized_test)
predictions = np.argmax(logits, axis=-1)
print("accuracy:", accuracy_score(tokenized_test['label'], predictions))
print("precision:", precision_score(tokenized_test['label'], predictions))
print("recall:", recall_score(tokenized_test['label'], predictions))
print("f1:", f1_score(tokenized_test['label'], predictions))
print("auc_roc:", roc_auc_score(tokenized_test['label'], torch.softmax(torch.tensor(logits), dim=-1).cpu().numpy()[:, 1]))
print("auc_pr:", average_precision_score(tokenized_test['label'], torch.softmax(torch.tensor(logits), dim=-1).cpu().numpy()[:, 1]))

/home/ubuntu/anaconda3/envs/foundmdl/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using device: cuda
shape of the dataframe before dropping columns (1348, 76)
shape of the dataframe after dropping columns (1348, 70)
Length of categorical and numerical columns: 59 11


Map: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 200/200 [00:00<00:00, 16180.79 examples/s]
Some weights of GPTNeoForSequenceClassification were not initialized from the model checkpoint at EleutherAI/gpt-neo-125M and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.752597,0.500000
2,No log,0.824968,0.500000
3,No log,0.958570,0.500000
4,No log,0.822555,0.500000
5,No log,0.683897,0.640000
6,No log,0.621370,0.670000
7,No log,0.722755,0.560000
8,0.639900,0.589487,0.700000
9,0.639900,0.762772,0.560000
10,0.639900,0.579727,0.700000


Checkpoint destination directory ./results/checkpoint-66 already exists and is non-empty. Saving will proceed but saved results may be invalid.
Checkpoint destination directory ./results/checkpoint-132 already exists and is non-empty. Saving will proceed but saved results may be invalid.
Checkpoint destination directory ./results/checkpoint-198 already exists and is non-empty. Saving will proceed but saved results may be invalid.
Checkpoint destination directory ./results/checkpoint-264 already exists and is non-empty. Saving will proceed but saved results may be invalid.
Checkpoint destination directory ./results/checkpoint-330 already exists and is non-empty. Saving will proceed but saved results may be invalid.
Checkpoint destination directory ./results/checkpoint-396 already exists and is non-empty. Saving will proceed but saved results may be invalid.
Checkpoint destination directory ./results/checkpoint-462 already exists and is non-empty. Saving will proceed but saved results ma

accuracy: 0.63
precision: 0.6382978723404256
recall: 0.6
f1: 0.6185567010309279
auc_roc: 0.6637
auc_pr: 0.6618486867227983
